# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import random

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

#TODO Samodzielna

mat = loadmat('MR_data.mat')

img1= mat['I_noisy1']
img2 = mat['I_noisy2']
img3 = mat['I_noisy3']
img4 = mat['I_noisy4']
img5 = mat['I_noisefree']


### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna

def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g  
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()
    

maska = fgaussian(5,0.7)
mesh(maska,5)




In [ ]:
def pixel_out(win,size,sigma):
     mask= fgaussian(size,sigma)
     A,B = win.shape
     x = A//2
     y = B//2
     pixel=0
     for i in range(A):
        for j in range(B):
            distance=np.sqrt((x-i)**2+(y-j)**2)
            fun_gaus= (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-(distance**2)/(2*sigma**2))
            pixel=pixel+fun_gaus*win[i,j]
     return pixel/mask.sum()

def convol(image,sigma,size):
     IConv = image.copy()
     A,B=IConv.shape
     step = size//2
     for i in range(step,A-step):
          for j in range(step,B-step//2):
               win = IConv[i-step:i+step+1,j-step:j+step+1]
               new_pixel=pixel_out(win,size,sigma)
               IConv[i,j]=new_pixel
     return IConv

f, ax1 = plt.subplots(1,2,figsize=(16,16))
ax1[0].imshow(img1,'gray')
ax1[0].set_title("Obraz oryginalny")
ax1[0].axis('off')
ax1[1].imshow(convol(img1,0.7,7), 'gray')
ax1[1].set_title("Obraz w wyniku konwolucji")
ax1[1].axis('off')

W wyniku konwolucji obraz stał się nieco bardziej rozmyty, krawędzie są mniej ostre, a białe plamki w tle bardziej niewyraźne

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def pixel_out_bilateral(win,sigma,delta_r):
    A,B = win.shape
    x = A//2
    y = B//2
    pixel=0
    norm=0
    for i in range(A):
        for j in range(B):
            diff=np.abs(win[x,y]-win[i,j])
            dist=np.sqrt(((x-i)**2)+((y-j)**2))
            gaus_fun= np.exp(-(dist**2)/(2*(sigma**2)))
            gamma_y= np.exp(-(diff**2)/(2*(delta_r**2)))
            pixel=pixel+gaus_fun*gamma_y*win[i,j]
            norm+=gaus_fun*gamma_y
    return pixel/norm

    #TODO Samodxielna
def bilateral(img,size,sigma,delta_r):
    IConv = img.copy()
    A,B=IConv.shape
    step = size//2
    for i in range(step,A-step):
        for j in range(step,B-step):
            win = IConv[i-step:i+step+1,j-step:j+step+1]
            new_pixel=pixel_out_bilateral(win,sigma,delta_r)
            IConv[i,j]=new_pixel
    return IConv


f, ax1 = plt.subplots(1,2,figsize=(16,16))
ax1[0].imshow(img1,'gray')
ax1[0].set_title("Onraz oryginalny")
ax1[0].axis('off')
ax1[1].imshow(bilateral(img1,5,5,20), 'gray')
ax1[1].set_title("Obraz w wyniku filtracji bilateralnej")
ax1[1].axis('off')

Obraz w wyniku filtracji bilaterlanej stał się wygładzony, artefakty w postaci białych kropek są niewidoczne (bardzo rozmyte,zlewają się)

In [ ]:
tab_image = [img1,img2, img3, img4]
ax1,ax2,ax3 = 0,0,0

def graphs(tab_image,tab_ax,tab_title):
    f, (tab_ax) = plt.subplots(1,np.size(tab_ax),figsize=(16,6))

    for i in range(np.size(tab_ax)):
        tab_ax[i].set_title(tab_title[i])
        tab_ax[i].imshow(tab_image[i], 'gray')
        tab_ax[i].axis('off')

for img in tab_image:
    graphs([img,convol(img,0.7,7),bilateral(img,5,5,20)],[ax1,ax2,ax3],["Obraz rzeczywisty","Wyniku konwolucji","Wynik filtracji bilaterlanej"])


Filtracja bilateralna wygładza obrazy, eliminiuje niedoskonałości najlepiej, przy zachowaniu ostrych krawędzi. Konwolucja powoduje powstanie mniej ostrych krawędzi w stosunku do obrazu oryginalnego.